# Write your own GNN module

This tutorial assumes that you already know [the basics of training a GNN for node classification](1_introduction.ipynb).  However, sometimes you would like to invent a new way of aggregating neighbor information or using the aggregation.

By the end of this tutorial you will be able to

* Understand DGL's message passing APIs.
* Implement GraphSAGE convolution module by your own.

In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

Using backend: pytorch


This tutorial assumes that you already know the pipeline of full-graph node classification.  If not, please refer to the [introduction](1_introduction.ipynb).

The following code for data loading and training loop is directly copied from the introduction tutorial.

In [2]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

def train(g, net):
    optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
    all_logits = []
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(200):
        # Forward
        logits = net(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that we should only compute the losses of the nodes in the training set,
        # i.e. with train_mask 1.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

Loading from cache failed, re-processing.
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


## Message passing and GNNs

DGL follows the *message passing paradigm* inspired by the Message Passing Neural Network proposed by [Gilmer et al.](https://arxiv.org/abs/1704.01212) Essentially, they found many GNN models can fit into the following framework:

$$
m_{u\to v}^{(l)} = M^{(l)}\left(h_v^{(l-1)}, h_u^{(l-1)}, e_{u\to v}^{(l-1)}\right)
$$

$$
m_{v}^{(l)} = \sum_{u\in\mathcal{N}(v)}m_{u\to v}^{(l)}
$$

$$
h_v^{(l)} = U^{(l)}\left(h_v^{(l-1)}, m_v^{(l)}\right)
$$

where DGL calls $M^{(l)}$ the *message function*, $\sum$ the *reduce function* and $U^{(l)}$ the *update function*.  Note that $\sum$ here can represent any function and is not necessarily a summation.

For example, the [GraphSAGE convolution (Hamilton et al., 2017)](https://cs.stanford.edu/people/jure/pubs/graphsage-nips17.pdf) takes the following mathematical form:

$$
h_{\mathcal{N}(v)}^k\leftarrow \text{Average}\{h_u^{k-1},\forall u\in\mathcal{N}(v)\}
$$

$$
h_v^k\leftarrow \text{ReLU}\left(W^k\cdot \text{CONCAT}(h_v^{k-1}, h_{\mathcal{N}(v)}^k) \right)
$$

You can see that message passing is directional: the message sent from one node $u$ to other node $v$ is not necessarily the same as the other message sent from node $v$ to node $u$ in the opposite direction.

Although DGL has builtin support of GraphSAGE via [`dgl.nn.SAGEConv`](https://docs.dgl.ai/api/python/nn.pytorch.html#sageconv), here is how you can implement GraphSAGE convolution in DGL by your own.

In [3]:
import dgl.function as fn

class SAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model.
    
    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)
    
    def forward(self, g, h):
        """Forward computation
        
        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        """
        with g.local_scope():
            g.ndata['h'] = h
            # update_all is a message passing API.
            g.update_all(message_func=fn.copy_u('h', 'm'), reduce_func=fn.mean('m', 'h_N'))
            h_neigh = g.ndata['h_N']
            h_total = torch.cat([h, h_neigh], dim=1)
            return self.linear(h_total)

The central piece in this code is the [`g.update_all`](https://docs.dgl.ai/generated/dgl.DGLGraph.update_all.html#dgl.DGLGraph.update_all) function, which gathers and averages the neighbor features. There are three concepts here:
* Message function `fn.copy_u('h', 'm')` that copies the node feature under name `'h'` as *messages* sent to neighbors.
* Reduce function `fn.mean('m', 'h_neigh')` that averages all the received messages under name `'m'` and saves the result as a new node feature `'h_N'`.
* `update_all` tells DGL to trigger the message and reduce functions for all the nodes and edges.

Afterwards, you can stack your own GraphSAGE convolution layers to form a multi-layer GraphSAGE network.

<div class="alert alert-info">
    <b>Note</b>: the GraphSAGE model here is only a demonstration of how you could write a neural network module; the hyperparameters and architecture choices are not tuned so the performance here may be inferior to what the paper actually reported.  For an accurate reproduction, please refer to <a href=https://github.com/dmlc/dgl/blob/master/examples/pytorch/graphsage/train_full.py>our example</a>.
</div>

In [4]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats)
        self.conv2 = SAGEConv(h_feats, num_classes)
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
    
model = Model(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.949, val acc: 0.058 (best 0.058), test acc: 0.072 (best 0.072)
In epoch 5, loss: 1.879, val acc: 0.394 (best 0.394), test acc: 0.381 (best 0.381)
In epoch 10, loss: 1.747, val acc: 0.600 (best 0.660), test acc: 0.593 (best 0.647)
In epoch 15, loss: 1.542, val acc: 0.606 (best 0.660), test acc: 0.601 (best 0.647)
In epoch 20, loss: 1.270, val acc: 0.656 (best 0.660), test acc: 0.652 (best 0.647)
In epoch 25, loss: 0.962, val acc: 0.700 (best 0.700), test acc: 0.695 (best 0.695)
In epoch 30, loss: 0.662, val acc: 0.736 (best 0.736), test acc: 0.717 (best 0.717)
In epoch 35, loss: 0.415, val acc: 0.758 (best 0.758), test acc: 0.748 (best 0.747)
In epoch 40, loss: 0.244, val acc: 0.766 (best 0.768), test acc: 0.751 (best 0.748)
In epoch 45, loss: 0.141, val acc: 0.766 (best 0.768), test acc: 0.761 (best 0.748)
In epoch 50, loss: 0.083, val acc: 0.766 (best 0.768), test acc: 0.765 (best 0.748)
In epoch 55, loss: 0.052, val acc: 0.768 (best 0.768), test acc: 0.764 (best 0

## More customization

In DGL, we provide many built-in message and reduce functions under the `dgl.function` package.  You can find more details in [the API doc](https://docs.dgl.ai/api/python/function.html).

These APIs allow one to quickly implement new graph convolution modules. For example, the following implements a new `SAGEConv` that aggregates neighbor representations using a weighted average.  Note that `edata` member can hold edge features which can also take part in message passing.

In [5]:
class WeightedSAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model with edge weights.
    
    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    def __init__(self, in_feat, out_feat):
        super(WeightedSAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)
    
    def forward(self, g, h, w):
        """Forward computation
        
        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        w : Tensor
            The edge weight.
        """
        with g.local_scope():
            g.ndata['h'] = h
            g.edata['w'] = w
            g.update_all(message_func=fn.u_mul_e('h', 'w', 'm'), reduce_func=fn.mean('m', 'h_neigh'))
            h_neigh = g.ndata['h_neigh']
            h_total = torch.cat([h, h_neigh], dim=1)
            return self.linear(h_total)

Because the graph in this dataset does not have edge weights, we manually assign all edge weights to one in the `forward()` function of the model.  You can replace it with your own edge weights.

In [6]:
class Net(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Net, self).__init__()
        self.conv1 = WeightedSAGEConv(in_feats, h_feats)
        self.conv2 = WeightedSAGEConv(h_feats, num_classes)
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat, torch.ones(g.num_edges()).to(g.device))
        h = F.relu(h)
        h = self.conv2(g, h, torch.ones(g.num_edges()).to(g.device))
        return h
    
net = Net(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, net)

In epoch 0, loss: 1.952, val acc: 0.114 (best 0.114), test acc: 0.103 (best 0.103)
In epoch 5, loss: 1.877, val acc: 0.298 (best 0.298), test acc: 0.299 (best 0.299)
In epoch 10, loss: 1.740, val acc: 0.458 (best 0.458), test acc: 0.452 (best 0.452)
In epoch 15, loss: 1.534, val acc: 0.556 (best 0.556), test acc: 0.562 (best 0.562)
In epoch 20, loss: 1.266, val acc: 0.616 (best 0.616), test acc: 0.624 (best 0.624)
In epoch 25, loss: 0.962, val acc: 0.692 (best 0.692), test acc: 0.704 (best 0.704)
In epoch 30, loss: 0.666, val acc: 0.712 (best 0.712), test acc: 0.728 (best 0.728)
In epoch 35, loss: 0.423, val acc: 0.742 (best 0.742), test acc: 0.749 (best 0.749)
In epoch 40, loss: 0.253, val acc: 0.740 (best 0.742), test acc: 0.765 (best 0.749)
In epoch 45, loss: 0.149, val acc: 0.754 (best 0.754), test acc: 0.771 (best 0.771)
In epoch 50, loss: 0.090, val acc: 0.754 (best 0.756), test acc: 0.777 (best 0.774)
In epoch 55, loss: 0.057, val acc: 0.754 (best 0.756), test acc: 0.778 (best 0

## Even more customization by user-defined function

DGL allows user-defined message and reduce function for the maximal expressiveness. Here is a user-defined message function that is equivalent to `fn.u_mul_e('h', 'w', 'm')`.

In [7]:
def u_mul_e_udf(edges):
    return {'m' : edges.src['h'] * edges.data['w']}

`edges` has three members: `src`, `data` and `dst`, representing the source node feature, edge feature, and destination node feature for all edges.

You can also write your own reduce function.  For example, the following is equivalent to the builtin `fn.sum('m', 'h')` function that sums up the incoming messages:

In [8]:
def sum_udf(nodes):
    return {'h': nodes.mailbox['m'].sum(1)}

In short, DGL will group the nodes by their in-degrees, and for each group DGL stacks the incoming messages along the second dimension.  One can then perform a reduction along the second dimension to aggregate messages.

For more details on customizing message and reduce function with user-defined function, please refer to the [API reference](https://docs.dgl.ai/api/python/udf.html#apiudf).

## Best practice of writing custom GNN modules

DGL recommends the following practice ranked by preference:

* Use `dgl.nn` modules.
* Use `dgl.nn.functional` functions which contain lower-level complex operations such as [computing a softmax for each node over incoming edges](https://TODO).
* Use `update_all` with builtin message and reduce functions.
* Use user-defined message or reduce functions.

## What's next?

* If you would like to scale to large graph (please refer to [this tutorial](L1_large_node_classification.ipynb) first to get an idea of how large graph training works), you will need to [change your GNN module implementation accordingly](L4_message_passing.ipynb).
* [Writing Efficient Message Passing Code](https://docs.dgl.ai/guide/message-efficient.html)